In [7]:
# a) Import required libraries
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.applications import VGG16
from tensorflow.keras.optimizers import Adam
import os

In [9]:
# ==========================================================
# Step 1: Load the Pre-trained CNN Model (VGG16 without top)
# ==========================================================
base = VGG16(
    weights=r'C:\assignement\sem7\dataset\Object Detection(Ass6)\vgg16_weights.h5',
    include_top=False,
    input_shape=(224,224,3)
)
print("✅ Pre-trained VGG16 model loaded successfully.")

✅ Pre-trained VGG16 model loaded successfully.


In [10]:

# ---- Freeze base model ----
for layer in base.layers:
    layer.trainable = False


In [11]:
train_gen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2
)

In [13]:
selected_classes = ['airplanes', 'ant']

train_data = train_gen.flow_from_directory(
    r"C:\assignement\sem7\dataset\Object Detection(Ass6)\caltech-101-img", target_size=(224,224),
    batch_size=32, subset='training', classes=selected_classes
)

val_data = train_gen.flow_from_directory(
    r"C:\assignement\sem7\dataset\Object Detection(Ass6)\caltech-101-img", target_size=(224,224),
    batch_size=32, subset='validation', classes=selected_classes
)


Found 674 images belonging to 2 classes.
Found 168 images belonging to 2 classes.


In [14]:

# ---- Add Classifier on top of base model ----
model = Sequential([
    base,
    Flatten(),
    Dense(256, activation='relu'),
    Dropout(0.2),
    Dense(256, activation='relu'),
    Dropout(0.4),
    Dense(train_data.num_classes, activation='softmax')
])

# ---- Compile (train only new layers) ----
model.compile(
    optimizer=Adam(),
    loss="categorical_crossentropy",
    metrics=["accuracy"]
)

In [15]:

# ---- Train ----
model.fit(train_data, validation_data=val_data, epochs=5)


Epoch 1/5
22/22 ━━━━━━━━━━━━━━━━━━━━ 121s 5s/step - accuracy: 0.9421 - loss: 0.3566 - val_accuracy: 0.9762 - val_loss: 0.0485
Epoch 2/5
22/22 ━━━━━━━━━━━━━━━━━━━━ 126s 6s/step - accuracy: 0.9941 - loss: 0.0260 - val_accuracy: 1.0000 - val_loss: 5.0456e-06
Epoch 3/5
22/22 ━━━━━━━━━━━━━━━━━━━━ 135s 6s/step - accuracy: 0.9985 - loss: 0.0049 - val_accuracy: 1.0000 - val_loss: 2.5683e-06
Epoch 4/5
22/22 ━━━━━━━━━━━━━━━━━━━━ 145s 7s/step - accuracy: 0.9985 - loss: 0.0100 - val_accuracy: 1.0000 - val_loss: 4.2953e-04
Epoch 5/5
22/22 ━━━━━━━━━━━━━━━━━━━━ 190s 9s/step - accuracy: 0.9970 - loss: 0.0209 - val_accuracy: 1.0000 - val_loss: 4.0446e-08


In [18]:
# ==========================================================
# Step 7: Fine-tune (unfreeze top layers)
# ==========================================================
# Unfreeze last 4 convolutional layers
for layer in base.layers[-4:]:
    layer.trainable = True

# Re-compile with smaller learning rate
model.compile(optimizer=Adam(learning_rate=1e-5),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

print("🔄 Fine-tuning top layers...")

fine_tune_history = model.fit(
    train_data,
    validation_data=val_data,
    epochs=5
)

# ==========================================================
# Step 8: Evaluate and Save the model
# ==========================================================
loss, acc = model.evaluate(val_data)
print(f"\n✅ Final Validation Accuracy: {acc*100:.2f}%")

model.save("vgg16_transfer_learning_final.h5")
print("✅ Model saved successfully as vgg16_transfer_learning_final.h5")

🔄 Fine-tuning top layers...
Epoch 1/5
22/22 ━━━━━━━━━━━━━━━━━━━━ 144s 7s/step - accuracy: 1.0000 - loss: 2.9464e-04 - val_accuracy: 1.0000 - val_loss: 2.7531e-07
Epoch 2/5
22/22 ━━━━━━━━━━━━━━━━━━━━ 153s 7s/step - accuracy: 0.9985 - loss: 0.0011 - val_accuracy: 1.0000 - val_loss: 4.2369e-06
Epoch 3/5
22/22 ━━━━━━━━━━━━━━━━━━━━ 152s 7s/step - accuracy: 1.0000 - loss: 7.2887e-06 - val_accuracy: 1.0000 - val_loss: 1.7363e-05
Epoch 4/5
22/22 ━━━━━━━━━━━━━━━━━━━━ 155s 7s/step - accuracy: 1.0000 - loss: 2.0353e-04 - val_accuracy: 1.0000 - val_loss: 2.3345e-07
Epoch 5/5
22/22 ━━━━━━━━━━━━━━━━━━━━ 156s 7s/step - accuracy: 1.0000 - loss: 3.9616e-05 - val_accuracy: 1.0000 - val_loss: 4.2575e-09
6/6 ━━━━━━━━━━━━━━━━━━━━ 26s 4s/step - accuracy: 1.0000 - loss: 4.2575e-09



✅ Final Validation Accuracy: 100.00%
✅ Model saved successfully as vgg16_transfer_learning_final.h5
